# Introduction

In this Jupyter notebook, I will be covering how I have used the amazing [XLSXWriter](https://xlsxwriter.readthedocs.io/).

While there is also [openpyxl](https://openpyxl.readthedocs.io/en/stable/), I really liked XLSXWriter much better.

While there are MANY ways to use this, I am going to cover how I use it to:
- read in an excel file
- create a new excel file
- some formatting
- centering columns

Be sure to [read the docs](https://xlsxwriter.readthedocs.io/) for additional things not covered here.

One last thing - the structure of the code here is for educational purposes. If this were a real program, I would structure this a lot differently.

# Setup

You will need the following to be installed:
- python 3
- pandas
- xlsxwriter
- xlrd

`pip3 install pandas`

`pip3 install XlsxWriter`

`pip3 install xlrd`

If you don't already have a [virtual environment](https://github.com/ProsperousHeart/cheatsheets/blob/master/Tools/VirtualEnvironments.md) created, please do so now. You can even [use my cheat sheet](https://github.com/ProsperousHeart/cheatsheets/blob/master/Processes/virtualenvs.md). Inside this virtual environment is where you should install **pandas** and **xlsxwriter** ... The reason for this is not every project can use some of the later versions of packages, so you want to ensure you don't cause issues later down the line.

In this walkthrough, I also provide a CSV file `AirQualityUCI.csv` that will allow us to populate data in the 1st tab. This comes from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Air+Quality) and covers air quality from March 2004 to February 2005. While we will not necessarily be using this data for any other purpose other than how to use XLSXWriter, this is what the data entails:
- 9358 instances of hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an air quality chemical multisensor device
- device was located in a significantly polluted area at road level in an Italian city
- Ground Truth hourly averaged concentrations for:
    1. CO2
    2. SNO - tin oxide
    3. NMHC - Non Metanic Hydrocarbons in micrograms
    4. C6H6 - Benzene (micrograms)
    5. TIO2 - titanium dioxide
    6. NOX - Total Nitrogen Oxides (ppb)
    7. WOX - Tungsten oxide
    8. NO2 - Nitrogen Dioxide (microgram)
- includes date & time

The second file I will use to show how you can create multiple tabs will be the `ionosphere.data.csv` file. This is from the Johns Hopkins University Ionosphere database, thanks to [Vince Sigillito](mailto:vgs@aplcen.apl.jhu.edu) in 1989. It is the classification of radar returns from the ionosphere using neural networks. It contains an array of 16 high-frequency antennas with a total transmitted power of 6.4 kilowatts, with the targets being free electrons in the ionosphere.

# Code

The first step you need is data. In whatever format you get it, plug it into a pandas DataFrame. Whether from MongoDB, hard coded, or even another XLSX file it really doesn't matter.

For this example, we'll use the XLSX file mentioned above.

## Reading In Files With Pandas

In this section, we will cover reading in files, either CSV or XLSX. For this example, I have only provided a CSV file. If you save it as an XLSX you will see any differences when reading in.

In [9]:
# add your imports
import pandas as pd
import xlsxwriter # yes, install was XlsxWriter but import all lower case

FILE1 = 'files/AirQualityUCI.csv' # note that reading in XLSX will provide different information when reading in a date
FILE2 = 'files/ionosphere.data.csv' # to be used in 2nd tab

In [10]:
def read_file(filename_str:str, file_type:int=0):
    """
    This function takes in the path of a CSV file, and an integer (0 for CSV, 1 for XLSX)
    and returns a DataFrame of the data.
    
    """
    if file_type not in [0, 1]:
        return (True, 'File type not accurately provided.', None)
    try:
        
        if file_type == 0:
            # ================================================================================
            # read in CSV file using Pandas
            # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
            # ================================================================================
            if filename_str[-4:] == '.csv':
                new_df = pd.read_csv(filename_str)
            else:
                return (True, 'CSV file does not have proper extension.', None)
        else:
            # ================================================================================
            # read in XLSX file using Pandas
            # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
            # ================================================================================
            if filename_str[-5:] == '.xlsx':
                new_df = pd.read_excel(filename_str)
            else:
                return (True, 'XLSX file does not have proper extension.', None)
    except FileNotFoundError as e:
        print("File not found. Please ensure the location and filename are correct.")
        new_df = None
    return (False, None, new_df)

In [11]:
err_bool, err_msg, df1 = read_file(FILE1, 0)
if err_bool == True:
    # exit() # you will need to reconnect the kernel if you hit this
    print(err_msg)
else:
    print("File successfully read and DataFrame created.")

File successfully read and DataFrame created.


In [12]:
df1.head()

,DATE,TIME,CO2,SNO,NMHC,C6H6,TIO2,NOX,WOX,NO2,WO2,O3,TEMP,RH,AH
0,10/03/2004,18.00.00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,10/03/2004,19.00.00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,10/03/2004,20.00.00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,10/03/2004,21.00.00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,10/03/2004,22.00.00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [13]:
err_bool, err_msg, df2 = read_file(FILE2, 0)
if err_bool == True:
    # exit() # you will need to reconnect the kernel if you hit this
    print(err_msg)
else:
    print("File successfully read and DataFrame created.")

File successfully read and DataFrame created.


In [14]:
df2.head()

,C00,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [15]:
# while Python has it's own garbage collection, I do my best to clean up after myself
# do your best not to leave any loopholes where possibly memory leaks may happen
del err_bool
del err_msg

## Writing & Saving To XLSX Files

Now that we have read in data and have a DataFrame, we can now create our XLSX file.

We did not need to read in from a file, we just needed some data in a DataFrame.

A DataFrame is **not** required to do this either. I just like it better.

It allows us to use a very special feature for XlsxWriter as outlined [here](https://xlsxwriter.readthedocs.io/working_with_pandas.html), including but not limited to:
- formatting date & datetime
- conversion of string to particular datasets (more info [here](https://xlsxwriter.readthedocs.io/workbook.html#Workbook))

<div class="alert alert-success"><b>NOTE:</b><br>Sometimes when writing to XLSX files, the output is not as you expect. <br>Likely you need to add the <b>'strings_to_formulas': False</b> as one of the options (shown below).</div>

Once you've created the writer and called the workbook, we can either add the data first or add properties to the workbook (or each worksheet). In this example I will show you what each looks like.

In [16]:
def create_writer(file:str, engine:str='xlsxwriter', date_format:str=None, datetime_format:str=None, options:dict=None, mode:str='w'):
    """
    This function takes in a filename string, type of engine, and optional parameters.
    Returns the ExcelWriter created with input variables.
    
    When creating an Excel writer with pandas, there are several options. Please see:
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html
    
    """
    
    writer = pd.ExcelWriter(file, engine='xlsxwriter', options={'strings_to_formulas': False})
    return writer

In [17]:
# create list of sheet names - max 30 characters
sheets = ["Air Quality", "Ionosphere"]
data_list = [df1, df2]

In [18]:
# create setup for no formatting - just straight to XLSX file with data
writerE = create_writer('files/NewFile_NoFormatting.xlsx')
workbookE = writerE.book

In [19]:
# write to each sheet
for idx, df in enumerate(data_list):
    sheetname = sheets[idx][:31] # sheetnames must be 30 characters or less
    
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html
    df.to_excel(writerE, sheet_name=sheetname, index=False) # index False will not write row names

In [20]:
# checking sheet names
for ws in workbookE.worksheets():
    print(ws.get_name())

Air Quality
Ionosphere


In [21]:
# whenever you want to keep changes, you must do this or the file won't be created
# save and close both seem to work - documentation says close for your workbook
writerE.save()
# writerE.close()
del workbookE
del writerE

## Formatting

As mentioned before, there are a LOT of options with which you can add formatting.

We're going to use a little bit of everything in the upcoming code.

### Workbook Formatting

Regarding your [workbook](https://xlsxwriter.readthedocs.io/workbook.html), you have the ability to set:
- workbook properties (title, subject, author, etc) & even custom properties
- window size
- different options like storing workbook data in temp files, converting strings to numbers, etc
- formatting on all worksheets at once
- tab ratio between tabs and horizontal slider
- variables in a workbook (define_name)
- macros or functions

### Worksheet Formatting

This is where you'll see things including but not limited to:
- how Excel sees a cell (URL, string, number, date, etc)
- column width
- adding a text box
- creating a button
- conditional formatting
- data validation
- creating tables, graphs, sparklines
- adding comments
- hiding sheets
- merging cells
- adding filters
- freeze or split panes
- setting zoom
- setting reading from right to left
- change tab color
- add protection

### Formatting Class

The [Format Class](https://xlsxwriter.readthedocs.io/format.html#format) is what is used to format the cells, including:
- fonts
- colors
- patterns
- borders
- alignment
- number formatting

In [22]:
def add_formats(workbook, df, sheetname):
    """
    This function takes in a worksheet, DataFrame, & sheetname.
    
        https://xlsxwriter.readthedocs.io/working_with_pandas.html
        
    Returns Format class objects to be used to change XLSX file.
    
    """
    
    def add_gen_prop(frmt_item, ctr_align:bool=True, num_frmt:str=None):
        """
        This function adds general properties to all formats.
            
        EXAMPLES:
            worksheet.write       (0, 0, 'Foo', cell_format)
            worksheet.write_string(1, 0, 'Bar', cell_format)
            worksheet.write_number(2, 0, 3,     cell_format)
            worksheet.write_blank (3, 0, '',    cell_format)
            worksheet.set_row(0, 18, cell_format)
            worksheet.set_column('A:D', 20, cell_format)
        
        """
        frmt_item.set_text_wrap()
        frmt_item.set_align('vcenter')
        if ctr_align:
            frmt_item.set_align('center')
        return frmt_item
    
    gen_cells = workbook.add_format()
    gen_cells = add_gen_prop(gen_cells, True)  # I personally like everything centered
    
    hdr_cells = workbook.add_format({'bold':1, 'underline':1})
    hdr_cells = add_gen_prop(hdr_cells, True)
    
    datetime_cells = workbook.add_format({'bold':1, 'font_color':'blue', 'underline':1})
    # datetime_cells = workbook.add_format()
    # datetime_cells.set_bold()
    # datetime_cells.set_underline()
    datetime_cells = add_gen_prop(datetime_cells, True)
    
    # http://pbpython.com/improve-pandas-excel-output.html
    # ah_cells = workbook.add_format({'num_format': '.00%'}) # - used to show percents
    ah_cells = workbook.add_format({'num_format': '.0000'}) # - used to show percents
    ah_cells = add_gen_prop(ah_cells, True)
    
    co2_cells = workbook.add_format({'num_format': '.00'}) # - used to show percents
    co2_cells = add_gen_prop(co2_cells, True)
    
    col_widths = get_col_widths(df)
    
    return (hdr_cells, datetime_cells, gen_cells, ah_cells, co2_cells)
    # return (hdr_cells, datetime_cells, gen_cells)

In [23]:
def add_cond_fmtg(wb, num_rows:int, ws, col:str, prcnt_goal_A:float=2.0, prcnt_goal_B:float=4.0):
    """
    Adds conditional formatting to requested columns. Nothing to return.

    """
    # tell xlsxwriter which rows you want to include (here all data)
    color_range = "{}2:{}{}".format(col, col, num_rows+1)

    # determine the properties of the cell if it hits the mark
    lower_bnd = wb.add_format({'bold': 1, 'bg_color': '#ffd5cc'})
    upper_bnd = wb.add_format({'bold': 1, 'bg_color': '#e1f2ea'})

    # add conditional formatting to the WS
    ws.conditional_format(color_range, {'type': 'cell', 'criteria': 'less than', 'value': prcnt_goal_A, 'format': lower_bnd})
    ws.conditional_format(color_range, {'type': 'cell', 'criteria': 'greater than', 'value': prcnt_goal_B, 'format': upper_bnd})

    # add another one for missing data
    equal = wb.add_format({'bold':1, 'bg_color':'#ffff00'})
    ws.conditional_format(color_range, {'type': 'cell', 'criteria': '==', 'value': -200.00, 'format': equal})

In [24]:
def get_col_widths(df:pd.DataFrame):
    """
    This takes in a DataFrame and returns a list of the sizes of each column for autofit.

    """
    cols = df.columns
    title_len_list = [len(col) for col in cols]
    col_len_list = []
    for col in cols:
        col_len_list.append(df[col].astype(str).map(len).max())
    # https://stackoverflow.com/a/40948355/10474024
    final_list = [max(val) + 4 for val in zip(title_len_list, col_len_list)]
    return final_list

In [25]:
# create the writer/book for the one to be formatted
# writerF = create_writer('files/NewFile_Formatting.xlsx', options={'strings_to_formulas': False, 'strings_to_numbers':True})
writerF = create_writer('files/NewFile_Formatting.xlsx', options={'strings_to_formulas': False})
workbookF = writerF.book

In [26]:
# sheets = ["Air Quality", "Ionosphere"]
# data_list = [df1, df2]
for idx, df in enumerate(data_list):
    sheet = sheets[idx][:31]
    number_rows = len(df.index)
    col_widths = get_col_widths(df)
    # print(col_widths)
    # https://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.DataFrame.to_excel.html
    df.to_excel(writerF, sheet_name=sheet, index=False)
    ws = writerF.sheets[sheet[:31]]
    
    hdr_cells, datetime_cells, gen_cells, ah_cells, co2_cells =  add_formats(workbookF, df, sheet)
    
    # ws.set_column("A2:A{}".format(number_rows+1), None, hdr_cells)
    ws.set_column("A2:B{}".format(number_rows+1), None, datetime_cells)
    
    # add conditional formatting to whichever columns you want
    if idx == 0:
        add_cond_fmtg(workbookF, number_rows, ws, 'C', 2.0, 4.0)
        add_cond_fmtg(workbookF, number_rows, ws, 'O', 0.9, 0.7)
        
        ws.set_column("A2:B{}".format(number_rows+1), None, datetime_cells)
        # ws.set_column("C2:N{}".format(number_rows+1), None, gen_cells)
        # ws.set_column("O2:O{}".format(number_rows+1), None, ah_cells)
        ws.set_column("C2:C{}".format(number_rows+1), None, co2_cells)
        ws.set_column("D2:N{}".format(number_rows+1), None, gen_cells)
        ws.set_column("O2:O{}".format(number_rows+1), None, ah_cells)
        
    else:
        ws.set_column("A2:AI{}".format(number_rows+1), None, gen_cells)
    
    # freeze panes - https://xlsxwriter.readthedocs.io/example_panes.html
    ws.freeze_panes(1, 0)
    
    # autofit - https://stackoverflow.com/a/37218180/10474024
    for width_idx, width in enumerate(col_widths):
        ws.set_column(width_idx, width_idx, width)

In [27]:
writerF.save()
writerF.close()
del workbookF
del writerF